In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import holidays
import itertools
from sklearn.linear_model import LinearRegression, Ridge
# from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, max_error, mean_absolute_error, r2_score

# 1. Import Data

In [ ]:
df_data_w_clusters = pd.read_csv('/content/drive/My Drive/MIE498 Thesis/Share-Bike-Station-Clustering-and-Usage-Prediction/station_data_w_clusters.csv', index_col=0)

In [ ]:
df_data_w_clusters.head()

,station_id,station_na,lat,lon,neighborhood_id,FSA_code,da_id,cluster
0,7021,Bay St / Albert St,43.653264,-79.382458,76,M5G,35204616.0,0
1,7160,King St W / Tecumseth St,43.643333,-79.405556,82,M5V,35204142.0,35
2,7012,Elizabeth St / Edward St (Bus Terminal),43.656026,-79.385327,76,M5G,35204607.0,16
3,7041,Edward St / Yonge St,43.656729,-79.382736,76,M5G,35204608.0,16
4,7275,Queen St W / James St,43.652276,-79.380701,76,M5G,35204616.0,34


In [ ]:
# Import ridership data
data_dir = '/content/drive/My Drive/MIE498 Thesis/0_Data'
merged_bike_data_2019 = pd.read_csv("{}/ridership_2019_with_bike_stations_info_20200930.csv".format(data_dir), header=0)
print(merged_bike_data_2019.shape)
merged_bike_data_2019['End Day of Year'] = merged_bike_data_2019.apply(lambda row: datetime.strptime(row['End Time'], "%Y-%m-%d %H:%M:%S").timetuple().tm_yday, axis=1)
print(merged_bike_data_2019.shape)
merged_bike_data_2019.head()

(2438720, 26)
(2438720, 27)


,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon,End Lat,End Lon,End Day of Year
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,2019.0,1.0,0.0,1.0,1.0,2019.0,1.0,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458,43.637922,-79.431734,1
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,2019.0,1.0,17.0,3.0,0.0,2019.0,1.0,17.0,3.0,0.0,3.0,1.0,43.653264,-79.382458,43.637922,-79.431734,3
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,2019.0,1.0,21.0,4.0,0.0,2019.0,1.0,22.0,4.0,0.0,4.0,1.0,43.653264,-79.382458,43.637922,-79.431734,4
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,2019.0,1.0,23.0,5.0,0.0,2019.0,1.0,23.0,5.0,0.0,5.0,1.0,43.653264,-79.382458,43.637922,-79.431734,5
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,2019.0,1.0,21.0,6.0,0.0,2019.0,1.0,22.0,6.0,0.0,6.0,1.0,43.653264,-79.382458,43.637922,-79.431734,6


In [ ]:
merged_data = merged_bike_data_2019.copy()
merged_data = merged_data.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='Start Station Id', right_on='station_id').drop('station_id', axis=1)
merged_data.rename({'cluster': 'Start Cluster'}, axis=1, inplace=True)
merged_data = merged_data.merge(df_data_w_clusters[['station_id', 'cluster']], how='left', left_on='End Station Id', right_on='station_id').drop('station_id', axis=1)
merged_data.rename({'cluster': 'End Cluster'}, axis=1, inplace=True)

In [ ]:
merged_data.isnull().sum()

Trip Id                0
Trip Duration          0
Start Station Id       0
Start Time             0
Start Station Name     0
End Station Id         0
End Time               0
End Station Name       0
User Type              0
Bike Id                0
Start Year             0
Start Month            0
Start Hour             0
Start Day of Week      0
Start Holiday          0
End Year               0
End Month              0
End Hour               0
End Day of Week        0
End Holiday            0
Start Day of Year      0
Start Week of Year     0
Start Lat              0
Start Lon              0
End Lat                0
End Lon                0
End Day of Year        0
Start Cluster          0
End Cluster            0
Start Day              0
End Day                0
Start Temp             0
Start Precipitation    0
End Temp               0
End Precipitation      0
dtype: int64

In [ ]:
merged_data.columns

Index(['Trip Id', 'Trip Duration', 'Start Station Id', 'Start Time',
       'Start Station Name', 'End Station Id', 'End Time', 'End Station Name',
       'User Type', 'Bike Id', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'Start Day of Year',
       'Start Week of Year', 'Start Lat', 'Start Lon', 'End Lat', 'End Lon',
       'End Day of Year', 'Start Cluster', 'End Cluster'],
      dtype='object')

In [ ]:
merged_data['Start Time'] = merged_data.apply(lambda row: datetime.strptime(row['Start Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
merged_data['Start Day'] = merged_data.apply(lambda row: row['Start Time'].day, axis=1)

In [ ]:
merged_data['End Time'] = merged_data.apply(lambda row: datetime.strptime(row['End Time'], "%Y-%m-%d %H:%M:%S"), axis=1)
merged_data['End Day'] = merged_data.apply(lambda row: row['End Time'].day, axis=1)

In [ ]:
merged_data.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type,Bike Id,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,End Year,End Month,End Hour,End Day of Week,End Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon,End Lat,End Lon,End Day of Year,Start Cluster,End Cluster,Start Day,End Day
0,4581278.0,1547.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,Annual Member,1296.0,2019.0,1.0,0.0,1.0,1.0,2019.0,1.0,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458,43.637922,-79.431734,1,0,32,1,1
1,4586979.0,1243.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,Annual Member,3168.0,2019.0,1.0,17.0,3.0,0.0,2019.0,1.0,17.0,3.0,0.0,3.0,1.0,43.653264,-79.382458,43.637922,-79.431734,3,0,32,3,3
2,4591641.0,1112.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,Annual Member,3512.0,2019.0,1.0,21.0,4.0,0.0,2019.0,1.0,22.0,4.0,0.0,4.0,1.0,43.653264,-79.382458,43.637922,-79.431734,4,0,32,4,4
3,4594328.0,1156.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,Annual Member,212.0,2019.0,1.0,23.0,5.0,0.0,2019.0,1.0,23.0,5.0,0.0,5.0,1.0,43.653264,-79.382458,43.637922,-79.431734,5,0,32,5,5
4,4596322.0,1068.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,Annual Member,3279.0,2019.0,1.0,21.0,6.0,0.0,2019.0,1.0,22.0,6.0,0.0,6.0,1.0,43.653264,-79.382458,43.637922,-79.431734,6,0,32,6,6


In [ ]:
merged_data.shape

(2438720, 31)

In [ ]:
# Import weather data
df_weather = pd.read_csv('/content/drive/My Drive/MIE498 Thesis/Share-Bike-Station-Clustering-and-Usage-Prediction/toronto_weather_2019.csv', index_col=None)
df_weather['Month'] = df_weather['Month'].astype("float64")
df_weather['Day'] = df_weather['Day'].astype("float64")
df_weather['Hour'] = df_weather['Hour'].astype("float64")
df_weather['Day of Week'] = df_weather['Day of Week'].astype("float64")
df_weather.head()

,Month,Day,Hour,Day of Week,Temperature (Celsius),Precipitation Amount (mm)
0,1.0,1.0,0.0,1.0,5.1,0.5
1,1.0,1.0,1.0,1.0,6.2,0.5
2,1.0,1.0,2.0,1.0,5.6,0.0
3,1.0,1.0,3.0,1.0,3.2,0.4
4,1.0,1.0,4.0,1.0,2.7,0.0


In [ ]:
merged_data = merged_data.merge(df_weather[['Month', 'Day', 'Hour', 'Temperature (Celsius)', 'Precipitation Amount (mm)']], how='left', left_on=['Start Month', 'Start Day', 'Start Hour'], right_on=['Month', 'Day', 'Hour']).drop(['Month', 'Day', 'Hour'], axis=1)
merged_data.rename({'Temperature (Celsius)': 'Start Temp', 'Precipitation Amount (mm)': 'Start Precipitation'}, axis=1, inplace=True)

In [ ]:
merged_data = merged_data.merge(df_weather[['Month', 'Day', 'Hour', 'Temperature (Celsius)', 'Precipitation Amount (mm)']], how='left', left_on=['End Month', 'End Day', 'End Hour'], right_on=['Month', 'Day', 'Hour']).drop(['Month', 'Day', 'Hour'], axis=1)
merged_data.rename({'Temperature (Celsius)': 'End Temp', 'Precipitation Amount (mm)': 'End Precipitation'}, axis=1, inplace=True)

In [ ]:
# check-out and check-in data split
checkout_data = merged_data[['Trip Id', 'Start Station Id', 'Start Time', 'Start Station Name', 'Start Year', 'Start Month', 'Start Hour',
       'Start Day of Week', 'Start Holiday', 'Start Day of Year', 'Start Week of Year', 'Start Lat', 'Start Lon', 'Start Cluster',
       'Start Temp', 'Start Precipitation']]
checkin_data = merged_data[['Trip Id', 'End Station Id', 'End Time', 'End Station Name', 'End Year', 'End Month',
       'End Hour', 'End Day of Week', 'End Holiday', 'End Lat', 'End Lon', 'End Day of Year', 'End Cluster', 
       'End Temp', 'End Precipitation']]

In [ ]:
# Training: Months 1-9, Testing: Months 10-12
def train_test_split(data, start_end_flag):
  training_data = data.loc[(data['{} Month'.format(start_end_flag)] >= 1) & (data['{} Month'.format(start_end_flag)] <= 9)]
  testing_data = data.loc[(data['{} Month'.format(start_end_flag)] >= 10) & (data['{} Month'.format(start_end_flag)] <= 12)]
  print(data.shape, training_data.shape, testing_data.shape)
  return training_data, testing_data

In [ ]:
def check_weekend(dayofweek):
  if dayofweek > 4:
    return 'weekend'
  else:
    return 'weekday'

# 2. Build prediction models for clusters (check-out numbers)

In [ ]:
checkout_data['weekday/weekend'] = checkout_data.apply(lambda row: check_weekend(row['Start Day of Week']), axis=1)
checkout_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Trip Id,Start Station Id,Start Time,Start Station Name,Start Year,Start Month,Start Hour,Start Day of Week,Start Holiday,Start Day of Year,Start Week of Year,Start Lat,Start Lon,Start Cluster,Start Temp,Start Precipitation,weekday/weekend
0,4581278.0,7021.0,2019-01-01 00:08:00,Bay St / Albert St,2019.0,1.0,0.0,1.0,1.0,1.0,1.0,43.653264,-79.382458,0,5.1,0.5,weekday
1,4586979.0,7021.0,2019-01-03 17:13:00,Bay St / Albert St,2019.0,1.0,17.0,3.0,0.0,3.0,1.0,43.653264,-79.382458,0,1.0,0.0,weekday
2,4591641.0,7021.0,2019-01-04 21:42:00,Bay St / Albert St,2019.0,1.0,21.0,4.0,0.0,4.0,1.0,43.653264,-79.382458,0,3.9,0.0,weekday
3,4594328.0,7021.0,2019-01-05 23:21:00,Bay St / Albert St,2019.0,1.0,23.0,5.0,0.0,5.0,1.0,43.653264,-79.382458,0,2.6,0.0,weekend
4,4596322.0,7021.0,2019-01-06 21:42:00,Bay St / Albert St,2019.0,1.0,21.0,6.0,0.0,6.0,1.0,43.653264,-79.382458,0,-4.6,0.0,weekend


In [ ]:
len(checkout_data['Start Day of Year'].unique())

365

In [ ]:
def fill_in_missing_combinations(data, start_end_flag):
  # Check missing combinations
  hours = np.arange(24)
  # dayofyear = data['{} Day of Year'.format(start_end_flag)].unique()
  weekday_weekend = ['weekday', 'weekend']
  clusters = np.arange(37)

  combinations = itertools.product(hours, weekday_weekend, clusters)
  df_combinations = data[['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag)]].to_numpy().astype('str')
  df_combinations = df_combinations.tolist()
  comb_list = []
  for comb in combinations:
    comb = [str(comb[0]), comb[1], str(comb[2])]
    if comb not in df_combinations:
      # print(comb)
      comb_list.append(comb)
  print('number of missing combinations:', len(comb_list))

  # Fill in missing combinations
  for comb in comb_list:
    hour = int(comb[0])
    weekday_weekend = comb[1]
    cluster = int(comb[2])
    new_row = {'{} Hour'.format(start_end_flag) : hour, 'weekday/weekend' : weekday_weekend, 
               '{} Cluster'.format(start_end_flag) : cluster, 'Trip Id': 0.000001}
    data = data.append(new_row, ignore_index = True)
    # dayofyear_list = data[(data['weekday/weekend'] == weekday_weekend)]['{} Day of Year'.format(start_end_flag)].to_numpy()
    # for day in dayofyear_list:
    #   temp = data[data['{} Day of Year'.format(start_end_flag)] == day]['{} Temp'.format(start_end_flag)]
    #   precipitation = data[data['{} Day of Year'.format(start_end_flag)] == day]['{} Precipitation'.format(start_end_flag)]
    #   new_row = {'{} Hour'.format(start_end_flag) : hour, 'weekday/weekend' : weekday_weekend, '{} Cluster'.format(start_end_flag) : cluster, 
    #              '{} Temp'.format(start_end_flag) : temp, '{} Precipitation'.format(start_end_flag) : precipitation, 'Trip Id': 0.000001}
    #   data = data.append(new_row, ignore_index = True)

  return data

In [ ]:
def preprocess_data(train_data, test_data, start_end_flag):
  train_data = train_data[['Trip Id', '{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag), '{} Temp'.format(start_end_flag), '{} Precipitation'.format(start_end_flag)]]
  train_data = train_data.groupby(by=['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag), '{} Temp'.format(start_end_flag), '{} Precipitation'.format(start_end_flag)]).count()
  print(train_data.head())
  train_data = train_data.reset_index(level=[0,1,2,3,4,5])
  train_data['{} Hour'.format(start_end_flag)] = train_data['{} Hour'.format(start_end_flag)].astype("int64")

  test_data = test_data[['Trip Id', '{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag), '{} Temp'.format(start_end_flag), '{} Precipitation'.format(start_end_flag)]]
  test_data = test_data.groupby(by=['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag), '{} Temp'.format(start_end_flag), '{} Precipitation'.format(start_end_flag)]).count()
  test_data = test_data.reset_index(level=[0,1,2,3,4,5])
  test_data['{} Hour'.format(start_end_flag)] = test_data['{} Hour'.format(start_end_flag)].astype("int64")

  train_data = fill_in_missing_combinations(train_data, start_end_flag)
  test_data = fill_in_missing_combinations(test_data, start_end_flag)

  if start_end_flag == 'Start':
    name = 'Number of Checkouts'
  else:
    name = 'Number of Checkins'

  train_data = train_data.sort_values(by=['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag), '{} Temp'.format(start_end_flag), '{} Precipitation'.format(start_end_flag)])
  train_data.rename({'Trip Id': name}, axis=1, inplace=True)
  train_data.reset_index(drop=True, inplace=True)

  test_data = test_data.sort_values(by=['{} Hour'.format(start_end_flag), 'weekday/weekend', '{} Cluster'.format(start_end_flag), '{} Day of Year'.format(start_end_flag), '{} Temp'.format(start_end_flag), '{} Precipitation'.format(start_end_flag)])
  test_data.rename({'Trip Id': name}, axis=1, inplace=True)
  test_data.reset_index(drop=True, inplace=True)

  print('Train Data Shape: ', train_data.shape)
  print('Test Data Shape: ', test_data.shape)

  return train_data, test_data

In [ ]:
checkout_train, checkout_test = train_test_split(checkout_data, 'Start')

(2438720, 17) (1970776, 17) (467944, 17)


In [ ]:
checkout_train, checkout_test = preprocess_data(checkout_train, checkout_test, 'Start')

                                                                                           Trip Id
Start Hour weekday/weekend Start Cluster Start Day of Year Start Temp Start Precipitation         
0.0        weekday         0             1.0                5.1       0.5                        1
                                         7.0               -5.9       0.0                        1
                                         14.0              -6.8       0.0                        1
                                         28.0              -12.7      0.0                        1
                                         38.0              -1.5       0.0                        1
number of missing combinations: 21
number of missing combinations: 74
Train Data Shape:  (154066, 7)
Test Data Shape:  (51089, 7)


In [ ]:
checkout_train.head()

,Start Hour,weekday/weekend,Start Cluster,Start Day of Year,Start Temp,Start Precipitation,Number of Checkouts
0,0,weekday,0,1.0,5.1,0.5,1.0
1,0,weekday,0,7.0,-5.9,0.0,1.0
2,0,weekday,0,14.0,-6.8,0.0,1.0
3,0,weekday,0,28.0,-12.7,0.0,1.0
4,0,weekday,0,38.0,-1.5,0.0,1.0


In [ ]:
max(checkout_train['Number of Checkouts']), max(checkout_test['Number of Checkouts']), min(checkout_train['Number of Checkouts']), min(checkout_test['Number of Checkouts'])

(363.0, 218.0, 1e-06, 1e-06)

In [ ]:
X_train = checkout_train[['Start Hour', 'weekday/weekend', 'Start Cluster', 'Start Temp', 'Start Precipitation']]
X_train.fillna(X_train.mean(), inplace=True)
X_train.isnull().sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Start Hour             0
weekday/weekend        0
Start Cluster          0
Start Temp             0
Start Precipitation    0
dtype: int64

In [ ]:
X_train = pd.get_dummies(data=X_train, columns=['Start Hour', 'weekday/weekend', 'Start Cluster'], drop_first=True).to_numpy()
# X_train = OneHotEncoder().fit_transform(X_train).toarray()
y_train = checkout_train[['Number of Checkouts']].to_numpy()
X_train.shape, y_train.shape

((154066, 62), (154066, 1))

In [ ]:
X_test = checkout_test[['Start Hour', 'weekday/weekend', 'Start Cluster', 'Start Temp', 'Start Precipitation']]
X_test.fillna(X_test.mean(), inplace=True)
X_test.isnull().sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Start Hour             0
weekday/weekend        0
Start Cluster          0
Start Temp             0
Start Precipitation    0
dtype: int64

In [ ]:
X_test = pd.get_dummies(data=X_test, columns=['Start Hour', 'weekday/weekend', 'Start Cluster'], drop_first=True).to_numpy()
y_test = checkout_test[['Number of Checkouts']].to_numpy()
X_test.shape, y_test.shape

((51089, 62), (51089, 1))

In [ ]:
y_train_log, y_test_log = np.log(y_train), np.log(y_test)

In [ ]:
# checkout_linreg = LinearRegression().fit(X_train, y_train)
checkout_linreg = LinearRegression().fit(X_train, y_train_log)

In [ ]:
checkout_linreg.coef_

array([[ 0.05853528, -0.17301614, -0.27116827, -0.4270598 , -0.73777744,
        -0.81596884, -0.42719537,  0.25034038,  0.967638  ,  1.63762383,
         1.36572272,  0.94719682,  1.01434754,  1.13590937,  1.13451996,
         1.09830719,  1.21165176,  1.42076474,  1.70603435,  1.56284032,
         1.35224415,  1.1296852 ,  0.85852354,  0.61338981,  0.36098241,
        -0.07388376, -1.26457909, -0.45242984, -0.48555658, -1.60339668,
        -0.36164424,  0.39407848, -1.85558933, -0.27943078, -0.08811422,
        -1.58445039, -2.18440741, -0.37354899, -1.96893806,  0.45590369,
        -1.1621491 ,  0.26615422, -2.14709046, -3.793732  , -1.28268732,
        -1.1926012 , -1.06145043, -1.51988865, -1.35296307, -1.84481558,
        -1.20693711, -1.08778604, -2.47845978, -2.08925826, -0.85419835,
         0.17896439, -0.42904131, -0.79479355, -0.09434913,  0.27974179,
         0.24625482, -2.11881431]])

In [ ]:
# import statsmodels.api as sm
# from scipy import stats
# X_train_3 = sm.add_constant(X_train)
# est = sm.OLS(y_train, X_train_3)
# est2 = est.fit()
# print(est2.summary())

In [ ]:
# X_train_2 = checkout_train[['Start Hour', 'weekday/weekend', 'Start Cluster']]
# X_train_2 = pd.get_dummies(data=X_train_2, columns=['Start Hour', 'weekday/weekend', 'Start Cluster'], drop_first=True)
# coeffs = pd.concat([pd.DataFrame(X_train_2.columns),pd.DataFrame(np.transpose(checkout_linreg.coef_))], axis = 1)
# coeffs.columns = ['Feature', 'LinReg Coef']
# print(['Feature', 'LinReg Coef'])
# for i, row in coeffs.iterrows():
#   print(row['Feature'], row['LinReg Coef'])

In [ ]:
y_train_pred = np.exp(checkout_linreg.predict(X_train))
y_test_pred = np.exp(checkout_linreg.predict(X_test))

In [ ]:
# R-squared values
print(checkout_linreg.score(X_train, y_train_log))
print(checkout_linreg.score(X_test, y_test_log))

0.6619678065952345
0.5440390115301967


In [ ]:
max(y_train), max(y_test)

(array([363.]), array([218.]))

In [ ]:
# MSE
print(mean_squared_error(y_train, y_train_pred, squared=True))
print(mean_squared_error(y_test, y_test_pred, squared=True))
# RMSE
print(mean_squared_error(y_train, y_train_pred, squared=False))
print(mean_squared_error(y_test, y_test_pred, squared=False))
# maximum residual error
print(max_error(y_train, y_train_pred))
print(max_error(y_test, y_test_pred))
# MAE
print(mean_absolute_error(y_train, y_train_pred))
print(mean_absolute_error(y_test, y_test_pred))
# R-squared
print(r2_score(y_train, y_train_pred))
print(r2_score(y_test, y_test_pred))

154.66630137497194
107.65265082312774
12.43649071784207
10.375579541554666
297.61615150176806
181.79964326986322
5.932381688148363
4.868733534265148
0.5672332774139913
0.42603884833792627


# 3. Compute a Transition Matrix

In [ ]:
data_train, data_test = train_test_split(merged_data, 'Start')

(2438720, 35) (1970776, 35) (467944, 35)


In [ ]:
data_train.shape

(1970776, 35)

In [ ]:
data_train['Start Cluster'].unique()

array([ 0, 35, 16, 34, 14, 30, 12,  2, 10, 31,  1, 33,  6, 29,  9, 32, 19,
       22, 15, 21,  8, 36,  3, 20, 23, 24,  4, 26, 17, 25,  5,  7, 11, 13,
       28, 18, 27])

In [ ]:
from_cluster_list = list(np.arange(0, 37))
df_transition_matrix = pd.DataFrame(data=np.zeros((37, 37)), columns = from_cluster_list, index=from_cluster_list)
df_transition_matrix = df_transition_matrix.astype("int")

for from_cluster in from_cluster_list:
  data_train_temp = data_train[data_train['Start Cluster'] == from_cluster]
  df_counts = pd.DataFrame(data_train_temp['End Cluster'].value_counts())
  
  # print(df_counts.head())
  to_cluster_list = list(df_counts.index)
  
  for to_cluster in to_cluster_list:
    cnt = df_counts.loc[to_cluster]['End Cluster']
    df_transition_matrix.loc[from_cluster, to_cluster] = cnt
  
  df_transition_matrix = df_transition_matrix.div(df_transition_matrix.sum(axis=1), axis=0)

In [ ]:
df_transition_matrix.replace(np.nan, 0, inplace=True)

In [ ]:
sum(df_transition_matrix.iloc[18])

1.0

In [ ]:
df_transition_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,0.088817,0.023390,0.025316,0.073562,0.007252,0.044845,0.109464,0.005082,0.098633,0.029017,0.000855,0.000451,0.009816,0.000601,0.031469,0.003307,0.062148,0.000122,0.000000,0.002809,0.000958,0.004387,0.000413,0.000892,0.002151,0.005768,0.016345,0.000094,0.000751,0.009563,0.084900,0.027382,0.013048,0.060345,0.083660,0.067287,0.005101
1,0.089650,0.082856,0.020639,0.079395,0.042090,0.045979,0.075763,0.006709,0.063499,0.018802,0.000427,0.001667,0.005128,0.002436,0.013332,0.004615,0.027733,0.000000,0.000000,0.000427,0.000214,0.000470,0.000171,0.000983,0.021537,0.047945,0.096402,0.000897,0.012392,0.002692,0.063413,0.013204,0.001966,0.015469,0.102213,0.037219,0.001667
2,0.069891,0.013799,0.086898,0.041862,0.012747,0.071857,0.131503,0.002949,0.082224,0.084414,0.002794,0.002156,0.014472,0.006175,0.062785,0.010384,0.067304,0.001414,0.000052,0.003536,0.000000,0.000604,0.000793,0.000949,0.000690,0.022216,0.006692,0.000673,0.005002,0.001794,0.051470,0.010470,0.006175,0.015524,0.063268,0.042621,0.001846
3,0.129491,0.035993,0.025833,0.092258,0.017434,0.049105,0.077483,0.016031,0.103201,0.015575,0.000196,0.000489,0.005121,0.000261,0.019097,0.001174,0.047165,0.000114,0.000082,0.000620,0.000424,0.000962,0.000196,0.000522,0.008627,0.012982,0.042452,0.001859,0.002593,0.007078,0.069801,0.015819,0.008073,0.031981,0.120929,0.035276,0.003702
4,0.060706,0.069123,0.040162,0.073033,0.090662,0.078799,0.111339,0.001789,0.063159,0.034727,0.000596,0.000464,0.004109,0.001657,0.033998,0.001591,0.058321,0.000066,0.000663,0.002054,0.000133,0.000133,0.000066,0.000199,0.004440,0.103652,0.027835,0.001789,0.013188,0.000663,0.053947,0.003380,0.000928,0.006230,0.037179,0.017894,0.001325
5,0.079505,0.019725,0.054262,0.053731,0.020013,0.098259,0.156706,0.002941,0.065981,0.047091,0.000500,0.000197,0.009006,0.001925,0.046181,0.002638,0.099882,0.000136,0.000030,0.001744,0.000030,0.000758,0.000212,0.000591,0.001501,0.019634,0.011386,0.000652,0.003654,0.001455,0.053458,0.007672,0.005352,0.008475,0.088966,0.033855,0.001895
6,0.109071,0.014444,0.045332,0.039070,0.011428,0.077166,0.146378,0.002522,0.047718,0.053686,0.001103,0.000795,0.017833,0.001598,0.072394,0.004779,0.083929,0.000279,0.000000,0.004263,0.000294,0.002092,0.000215,0.001354,0.000688,0.011306,0.005761,0.000129,0.001333,0.002680,0.066268,0.013678,0.005288,0.020807,0.082503,0.049223,0.002594
7,0.047853,0.010344,0.012329,0.117020,0.004075,0.015568,0.029986,0.139693,0.128931,0.003761,0.000104,0.000000,0.002194,0.000313,0.006164,0.000627,0.007418,0.000104,0.000000,0.000627,0.000522,0.000627,0.000104,0.000522,0.022673,0.004806,0.176784,0.000104,0.001254,0.003970,0.024240,0.007836,0.002612,0.066451,0.139484,0.020061,0.000836
8,0.114424,0.021311,0.027679,0.088210,0.010597,0.041666,0.064076,0.019468,0.105021,0.013018,0.000236,0.000083,0.005505,0.000461,0.015854,0.001949,0.032723,0.000024,0.000024,0.000650,0.000839,0.002918,0.000201,0.000543,0.005576,0.007891,0.039031,0.000130,0.000721,0.014034,0.069392,0.020862,0.008139,0.070715,0.152298,0.039917,0.003816
9,0.045938,0.005674,0.055292,0.014675,0.006779,0.034168,0.111540,0.000444,0.023208,0.129599,0.008146,0.005059,0.031035,0.007782,0.140274,0.034225,0.113067,0.003065,0.000057,0.010402,0.000194,0.001048,0.001903,0.004295,0.000513,0.014219,0.002632,0.000741,0.002267,0.002563,0.049903,0.012065,0.005560,0.016281,0.056214,0.047077,0.002096


# 4. Make check-in predictions
* check-out predictions x transition matrix = check-in predictions

In [ ]:
X_test_checkout = checkout_test[['Start Hour', 'weekday/weekend', 'Start Cluster', 'Start Temp', 'Start Precipitation']]
X_test_checkout.fillna(X_test_checkout.mean(), inplace=True)
X_test_checkout.isnull().sum()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Start Hour             0
weekday/weekend        0
Start Cluster          0
Start Temp             0
Start Precipitation    0
dtype: int64

In [ ]:
X_test_checkout.shape

(51089, 5)

In [ ]:
X_test_checkout.head()

,Start Hour,weekday/weekend,Start Cluster,Start Temp,Start Precipitation
0,0,weekday,0,16.3,0.0
1,0,weekday,0,19.5,0.0
2,0,weekday,0,10.1,0.0
3,0,weekday,0,11.0,0.0
4,0,weekday,0,14.5,0.0


In [ ]:
y_test_pred

array([[5.4254223 ],
       [6.54308407],
       [3.77415693],
       ...,
       [0.41798452],
       [0.32307652],
       [0.39422   ]])

In [ ]:
df_predictions = X_test_checkout.copy()
df_predictions['Check-out Predictions'] = y_test_pred
df_predictions['Check-out True Values'] = y_test

In [ ]:
df_predictions.head()

,Start Hour,weekday/weekend,Start Cluster,Start Temp,Start Precipitation,Check-out Predictions,Check-out True Values
0,0,weekday,0,16.3,0.0,5.425422,4.0
1,0,weekday,0,19.5,0.0,6.543084,5.0
2,0,weekday,0,10.1,0.0,3.774157,6.0
3,0,weekday,0,11.0,0.0,3.978317,11.0
4,0,weekday,0,14.5,0.0,4.882866,3.0


In [ ]:
min(df_predictions['Check-out Predictions'])

0.026606967695791767

In [ ]:
checkin_prediction_list = []

for idx, row in df_predictions.iterrows():
  checkout_prediction = row['Check-out Predictions']
  # print(checkout_prediction)
  cluster_n = row['Start Cluster']
  
  transition_list = np.array(df_transition_matrix.iloc[cluster_n])
  checkin_prediction = np.round(transition_list * checkout_prediction, 6)
  checkin_prediction_list.append(checkin_prediction)

df_predictions['Check-in Predictions'] = checkin_prediction_list

In [ ]:
df_predictions.shape

(51089, 8)

In [ ]:
df_predictions.head(10)

,Start Hour,weekday/weekend,Start Cluster,Start Temp,Start Precipitation,Check-out Predictions,Check-out True Values,Check-in Predictions
0,0,weekday,0,16.3,0.0,5.425422,4.0,"[0.481869, 0.126902, 0.137349, 0.399103, 0.039..."
1,0,weekday,0,19.5,0.0,6.543084,5.0,"[0.581136, 0.153044, 0.165644, 0.48132, 0.0474..."
2,0,weekday,0,10.1,0.0,3.774157,6.0,"[0.335209, 0.088278, 0.095546, 0.277633, 0.027..."
3,0,weekday,0,11.0,0.0,3.978317,11.0,"[0.353342, 0.093053, 0.100715, 0.292651, 0.028..."
4,0,weekday,0,14.5,0.0,4.882866,3.0,"[0.433681, 0.114211, 0.123614, 0.359191, 0.035..."
5,0,weekday,0,10.9,0.0,3.955097,6.0,"[0.351279, 0.09251, 0.100127, 0.290943, 0.0286..."
6,0,weekday,0,12.3,0.0,4.292866,6.0,"[0.381279, 0.100411, 0.108678, 0.31579, 0.0311..."
7,0,weekday,0,14.4,0.0,4.854367,7.0,"[0.43115, 0.113544, 0.122892, 0.357095, 0.0352..."
8,0,weekday,0,10.8,0.0,3.932014,4.0,"[0.349229, 0.09197, 0.099542, 0.289245, 0.0285..."
9,0,weekday,0,8.3,0.0,3.396731,5.0,"[0.301687, 0.07945, 0.085991, 0.249869, 0.0246..."


In [ ]:
# Checkin data
checkin_data['weekday/weekend'] = checkin_data.apply(lambda row: check_weekend(row['End Day of Week']), axis=1)
checkin_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Trip Id,End Station Id,End Time,End Station Name,End Year,End Month,End Hour,End Day of Week,End Holiday,End Lat,End Lon,End Day of Year,End Cluster,End Temp,End Precipitation,weekday/weekend
0,4581278.0,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,2019.0,1.0,0.0,1.0,1.0,43.637922,-79.431734,1,32,5.1,0.5,weekday
1,4586979.0,7233.0,2019-01-03 17:34:00,King / Cowan Ave - SMART,2019.0,1.0,17.0,3.0,0.0,43.637922,-79.431734,3,32,1.0,0.0,weekday
2,4591641.0,7233.0,2019-01-04 22:00:00,King / Cowan Ave - SMART,2019.0,1.0,22.0,4.0,0.0,43.637922,-79.431734,4,32,3.0,0.0,weekday
3,4594328.0,7233.0,2019-01-05 23:40:00,King / Cowan Ave - SMART,2019.0,1.0,23.0,5.0,0.0,43.637922,-79.431734,5,32,2.6,0.0,weekend
4,4596322.0,7233.0,2019-01-06 22:00:00,King / Cowan Ave - SMART,2019.0,1.0,22.0,6.0,0.0,43.637922,-79.431734,6,32,-4.9,0.0,weekend


In [ ]:
checkin_train, checkin_test = train_test_split(checkin_data, 'End')
checkin_train, checkin_test = preprocess_data(checkin_train, checkin_test, 'End')
y_checkin_test = checkin_test[['Number of Checkins']].to_numpy()
y_checkin_test.shape

(2438720, 16) (1970751, 16) (467969, 16)
                                                                                 Trip Id
End Hour weekday/weekend End Cluster End Day of Year End Temp End Precipitation         
0.0      weekday         0           1                5.1     0.5                      2
                                     4                1.3     0.0                      1
                                     7               -5.9     0.0                      1
                                     17              -9.3     0.0                      1
                                     24               4.7     0.0                      2
number of missing combinations: 16
number of missing combinations: 63
Train Data Shape:  (152466, 7)
Test Data Shape:  (50453, 7)


(50453, 1)

In [ ]:
# 10K data points takes ~14mins

In [ ]:
df_copy = df_predictions.copy()[['Start Hour', 'weekday/weekend', 'Start Cluster', 'Start Temp', 'Start Precipitation']]

In [ ]:
checkin_predictions_final = []
for i, row in df_copy.iterrows():
  if i % 1000 == 0:
    print(i)
  checkin_predictions_list = df_predictions.loc[(df_predictions['Start Hour'] == row[0]) & (df_predictions['weekday/weekend'] == row[1]) & (df_predictions['Start Cluster'] == row[2]) & (df_predictions['Start Temp'] == row[3]) & (df_predictions['Start Precipitation'] == row[4])]['Check-in Predictions'].to_numpy()[0]
  df_checkin_temp = pd.DataFrame(columns=np.arange(37))
  for i in range(37):
    df_checkin_temp = df_checkin_temp.append(pd.Series(checkin_predictions_list[i]), ignore_index=True)
  # compute column sums
  checkin_col_sums = list(df_checkin_temp.sum(axis=0).to_numpy())
  checkin_predictions_final += checkin_col_sums

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000


In [ ]:
checkin_predictions_final

[5.425423,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6.543085,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.7741570000000007,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 3.9783200000000005,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.882864,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 

In [ ]:
df_predictions_checkin['Check-in Predictions'] = checkin_predictions_final

ValueError: ignored

In [ ]:
# hours = np.arange(24)
# weekday_weekend = ['weekday', 'weekend']
# clusters = np.arange(37)
# combinations = itertools.product(hours, weekday_weekend)

# checkin_predictions_final = []
# for comb in combinations:
#   # obtain 37x37 check-in prediction matrix for the (hour, weekday/weekend) combination
#   checkin_predictions_list = df_predictions.loc[(df_predictions['Start Hour'] == comb[0]) & (df_predictions['weekday/weekend'] == comb[1])]['Check-in Predictions'].to_numpy()
#   df_checkin_temp = pd.DataFrame(columns=np.arange(37))
#   for i in range(37):
#     df_checkin_temp = df_checkin_temp.append(pd.Series(checkin_predictions_list[i]), ignore_index=True)
#   # compute column sums
#   checkin_col_sums = list(df_checkin_temp.sum(axis=0).to_numpy())
#   checkin_predictions_final += checkin_col_sums

# # Add to dataframe
# df_predictions_checkin = pd.DataFrame(data=checkin_test[['End Hour', 'weekday/weekend', 'End Cluster', 'End Day of Year', 'End Temp', 'End Precipitation']])
# df_predictions_checkin['Check-in True Values'] = y_checkin_test
# df_predictions_checkin['Check-in Predictions'] = checkin_predictions_final

ValueError: ignored

In [ ]:
y_checkin_test

array([[5.],
       [5.],
       [6.],
       ...,
       [1.],
       [1.],
       [1.]])

In [ ]:
df_predictions.head()

# 5 Evaluation

In [ ]:
def evaluation(y_true, y_pred):
  print('MSE: ', mean_squared_error(y_true, y_pred, squared=True)) # MSE
  print('RMSE: ', mean_squared_error(y_true, y_pred, squared=False)) # RMSE
  print('Maximum residual error: ', max_error(y_true, y_pred)) # maximum residual error
  print('MAE: ', mean_absolute_error(y_true, y_pred)) # MAE

In [ ]:
checkout_true = df_predictions['Check-out True Values'].to_numpy()
checkout_pred = df_predictions['Check-out Predictions'].to_numpy()
checkin_true = df_predictions['Check-in True Values'].to_numpy()
checkin_pred = df_predictions['Check-in Predictions'].to_numpy()

In [ ]:
print('Checkout evaluation:')
evaluation(checkout_true, checkout_pred)
print(min(checkout_true), max(checkout_true))

In [ ]:
print('Checkin evaluation:')
evaluation(checkin_true, checkin_pred)
print(min(checkin_true), max(checkin_true))